# Combining Balances and Removing Internal Transactions

**Goal:** Combine account balances from entities A, B, and C, but remove the effect of transactions *between* them.

**Convention:**
*   Positive (+) numbers: Assets (like Cash, Inventory), Expenses (like COGS, SG&A).
*   Negative (-) numbers: Liabilities (like Payables), Equity, Revenue (like Sales).
*   The total sum for each entity should be zero.

**Internal Transactions to Remove:**
1.  Sale from A to B (50,000). B still owes A.
2.  Profit A recorded on goods B still holds (4,000).
3.  Loan from A to C (100,000).
4.  Interest on the loan from A to C (5,000).

## 1. Setup: Account Balances for A, B, C

In [ ]:
import pandas as pd
import numpy as np
import math

pd.options.display.float_format = '{:,.2f}'.format

# --- Balances for A ---
data_a = {
    'Account': ['Cash', 'Receivable External', 'Receivable from B', 'Inventory', 'Loan to C', 'Fixed Assets',
                'Payable External', 'Equity', # Simplified Equity
                'Sales External', 'Sales to B', 'COGS', 'SG&A Expense', 'Interest Income from C'],
    'Amount': [50000, 30000, 50000, 120000, 100000, 500000, # Assets (+)
               -60000, -490000,                            # Liabilities/Equity (-)
               -550000, -50000, 380000, 150000, -5000]      # Revenue (-), Expense (+)
}
balances_a = pd.DataFrame(data_a).set_index('Account')

# --- Balances for B ---
data_b = {
    'Account': ['Cash', 'Receivable External', 'Inventory', 'Fixed Assets',
                'Payable External', 'Payable to A', 'Equity', # Simplified Equity
                'Sales External', 'COGS', 'SG&A Expense'],
    'Amount': [20000, 40000, 60000, 300000, # Assets (+)
               -30000, -50000, -230000,     # Liabilities/Equity (-)
               -400000, 250000, 110000]     # Revenue (-), Expense (+)
}
balances_b = pd.DataFrame(data_b).set_index('Account')

# --- Balances for C ---
data_c = {
    'Account': ['Cash', 'Receivable External', 'Inventory', 'Fixed Assets',
                'Payable External', 'Loan from A', 'Equity', # Simplified Equity
                'Sales External', 'COGS', 'SG&A Expense', 'Interest Expense to A'],
    'Amount': [10000, 30000, 40000, 200000, # Assets (+)
               -40000, -100000, -130000,     # Liabilities/Equity (-)
               -250000, 150000, 85000, 5000]  # Revenue (-), Expense (+)
}
balances_c = pd.DataFrame(data_c).set_index('Account')

# --- Verify totals are zero ---
print(f"A Total Check: {balances_a['Amount'].sum():,.2f}")
print(f"B Total Check: {balances_b['Amount'].sum():,.2f}")
print(f"C Total Check: {balances_c['Amount'].sum():,.2f}\n")

# --- Combine balances by just adding them up (Incorrect View) ---
combined_balances = pd.concat([
    balances_a.rename(columns={'Amount': 'A'}),
    balances_b.rename(columns={'Amount': 'B'}),
    balances_c.rename(columns={'Amount': 'C'})
], axis=1).fillna(0)

combined_balances['Simple Sum'] = combined_balances.sum(axis=1)
print("--- Combined Balances (Simple Sum - Incorrect View - Partial) ---")
# Show some key accounts affected by internal transactions
key_accounts = ['Receivable from B', 'Payable to A', 'Sales to B', 'COGS', 'Inventory', 'Loan to C', 'Loan from A', 'Interest Income from C', 'Interest Expense to A']
print(combined_balances.loc[combined_balances.index.isin(key_accounts)])
print(f"\nSimple Sum Total Check: {combined_balances['Simple Sum'].sum():,.2f}\n")

## 2. Solution: Rule-Based Removal of Internal Transactions

We define the internal transactions and rules to generate the necessary adjustments to cancel them out.

In [ ]:
print("\n--- Solution: Rule-Based Adjustments ---")

# --- Define Internal Transactions Info ---
internal_transactions = [
    {'Type': 'Sale', 'From': 'A', 'To': 'B', 'Amount': 50000, 'Cost': 30000, 'Is_Owed': True, 'Inventory Held By To': 10000},
    {'Type': 'Loan', 'From': 'A', 'To': 'C', 'Amount': 100000, 'Interest': 5000, 'Is_Owed': True},
]

# --- Define Rules for Adjustments ---
# Rules explain how to adjust accounts for each transaction type.
# Sign: +1 means add (like a Debit), -1 means subtract (like a Credit).
#       Remember: Adding to a negative number makes it less negative.
adjustment_rules = {
    'Sale': [
        # Cancel out the sale revenue recorded by 'From' entity
        {'Account': lambda t: f"Sales to {t['To']}", 'Sign': 1, 'Value': lambda t: t['Amount'], 'Reason': 'Remove internal sale revenue'},
        # Cancel out the cost recorded by 'To' entity for this purchase
        {'Account': 'COGS', 'Sign': -1, 'Value': lambda t: t['Amount'], 'Reason': 'Remove internal purchase cost'},

        # Adjust for profit on goods 'To' still holds
        # Calculation: Amount held * (Profit / Sale Amount)
        # Effect: Increase COGS (reduces overall profit), Decrease Inventory (value it at original cost)
        {'Account': 'COGS',      'Sign': 1, 'Value': lambda t: (t['Inventory Held By To'] * (t['Amount'] - t['Cost']) / t['Amount']) if t.get('Inventory Held By To', 0) > 0 and t['Amount'] > 0 else 0, 'Condition': lambda t: t.get('Inventory Held By To', 0) > 0, 'Reason': 'Remove profit on unsold goods'},
        {'Account': 'Inventory', 'Sign': -1,'Value': lambda t: (t['Inventory Held By To'] * (t['Amount'] - t['Cost']) / t['Amount']) if t.get('Inventory Held By To', 0) > 0 and t['Amount'] > 0 else 0, 'Condition': lambda t: t.get('Inventory Held By To', 0) > 0, 'Reason': 'Reduce inventory value by internal profit'},
    ],
    'Sale_Is_Owed': [
        # Cancel out the payable recorded by 'To' entity
        {'Account': lambda t: f"Payable to {t['From']}", 'Sign': 1, 'Value': lambda t: t['Amount'], 'Reason': 'Remove internal payable'},
        # Cancel out the receivable recorded by 'From' entity
        {'Account': lambda t: f"Receivable from {t['To']}", 'Sign': -1, 'Value': lambda t: t['Amount'], 'Reason': 'Remove internal receivable'},
    ],
    'Loan': [
        # Cancel out the loan payable recorded by 'To' entity
        {'Account': lambda t: f"Loan from {t['From']}", 'Sign': 1, 'Value': lambda t: t['Amount'], 'Reason': 'Remove internal loan payable'},
        # Cancel out the loan receivable recorded by 'From' entity
        {'Account': lambda t: f"Loan to {t['To']}", 'Sign': -1, 'Value': lambda t: t['Amount'], 'Reason': 'Remove internal loan receivable'},
        # Cancel out the interest income recorded by 'From' entity
        {'Account': lambda t: f"Interest Income from {t['To']}", 'Sign': 1, 'Value': lambda t: t['Interest'], 'Reason': 'Remove internal interest income'},
        # Cancel out the interest expense recorded by 'To' entity
        {'Account': lambda t: f"Interest Expense to {t['From']}", 'Sign': -1, 'Value': lambda t: t['Interest'], 'Reason': 'Remove internal interest expense'},
    ]
    # Note: Loan_Is_Owed could be separate if needed for principal vs interest.
}

# --- Generate Adjustments from Transactions and Rules ---
adjustments_list = []
for txn in internal_transactions:
    txn_type = txn['Type']
    rules = adjustment_rules.get(txn_type, [])

    if txn.get('Is_Owed'):
        rules.extend(adjustment_rules.get(f"{txn_type}_Is_Owed", []))

    for rule in rules:
        if 'Condition' in rule and not rule['Condition'](txn):
            continue

        account_val = rule['Account']
        account_name = account_val(txn) if callable(account_val) else account_val
        value = rule['Value'](txn)
        # Amount = Sign * Value. Positive means add to balance, Negative means subtract.
        amount = rule['Sign'] * value
        reason = rule.get('Reason', '')

        if abs(amount) > 1e-6:
            adjustments_list.append({'Account': account_name, 'Adjustment': amount, 'Reason': reason})

# --- Aggregate Adjustments by Account ---
adjustments_df = pd.DataFrame(adjustments_list).groupby('Account')['Adjustment'].sum().reset_index().set_index('Account')
print("\n--- Summary of Adjustments Generated ---")
print(adjustments_df)
print(f"Adjustments Total Check: {adjustments_df['Adjustment'].sum():,.2f}\n") # Should be 0

# --- Apply Adjustments to the Simple Sum ---
final_balances = combined_balances[['Simple Sum']].join(adjustments_df, how='outer').fillna(0)
# Rename for clarity
final_balances = final_balances.rename(columns={'Simple Sum': 'Combined Before Adjust', 'Adjustment': 'Internal Txn Adjustment'})
final_balances['Final Amount'] = final_balances['Combined Before Adjust'] + final_balances['Internal Txn Adjustment']

print("\n--- Balances After Applying Adjustments (Focus on Affected Accounts) ---")
# Show the effect on key accounts
print(final_balances.loc[final_balances.index.isin(key_accounts) | final_balances['Internal Txn Adjustment'] != 0])

print(f"\nFinal Total Check: {final_balances['Final Amount'].sum():,.2f}") # Should be 0

## 3. Conceptual Outline: Specific Account Matching

This approach relies on having very specific account names for all internal transactions.

1.  **Setup:** Ensure account names clearly identify the counterparty (e.g., 'Receivable from B', 'Payable to A', 'Sales to C', 'Loan from B').
2.  **Combine:** Add up all balances from A, B, C.
3.  **Match & Cancel:** Find pairs of accounts that represent the same internal transaction (e.g., 'Receivable from B' and 'Payable to A'). Their sum in the combined list should ideally be zero. Create adjustments to force them to zero.
4.  **Profit Adjustment:** The profit on goods not yet sold externally (like the 4,000 in our example) still needs a separate calculation and adjustment (Add 4,000 to COGS, Subtract 4,000 from Inventory). This isn't usually handled by simple account matching.

*Pros:* Conceptually simple for matching balances (like Loan A->C vs Loan C<-A).
*Cons:* Requires very strict and detailed account naming. Doesn't automatically handle profit adjustments. Can be hard to manage many specific accounts.

## 4. Evaluation

| Feature                 | Solution 2: Rule-Based      | Solution 3: Specific Acct Match (Conceptual) |
| :---------------------- | :-------------------------- | :------------------------------------------- |
| **Automation Level**    | High                        | Medium (High for matching, Low for profit adj.) |
| **Scalability**         | Good (add data/rules)       | Medium (Account list management, profit logic) |
| **Readability**         | Medium (rules logic)        | Medium (depends on account name clarity)     |
| **Maintainability**     | Good (central rules)        | Medium (Account list changes, profit logic)  |
| **Flexibility (New Txn)** | High (add new rule)         | Medium (new accounts + profit logic)         |
| **Error Prone**         | Low (if rules correct)      | Medium (mismatches, profit calculation)      |

**Conclusion:**

*   **Rule-Based (Solution 2):** Generally preferred for automation. It separates the *what* (transaction data) from the *how* (elimination rules). It handles calculations like the profit adjustment logically within its rules. It's more adaptable to different types of internal transactions.
*   **Specific Account Matching (Solution 3):** Can work if account structure is rigorously maintained for matching. Still needs separate logic for calculated adjustments like the profit on unsold goods.